Import Libraries

In [9]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import matplotlib.pyplot as plt
import random

print("All libraries imported successfully.")

All libraries imported successfully.


In [13]:
def load_env():
    """Load environment variables from a .env file."""
    env_file_path = '.env'
    try:
        with open(env_file_path) as f:
            for line in f:
                # Remove whitespace and skip comments
                line = line.strip()
                if line and not line.startswith('#'):
                    key, value = line.split('=', 1)
                    os.environ[key.strip()] = value.strip()  # Strip whitespace from both key and value
    except FileNotFoundError:
        print(f"Warning: {env_file_path} file not found.")

In [14]:
load_env()

In [5]:
# load the .env file variables
load_dotenv("../Spotify-Song-Recommendation/.env")

False

In [3]:
client_id = os.getenv("CLIENT_ID")
client_secret = os.environ.get("CLIENT_SECRET")

In [ ]:
print(client_id)

In [ ]:
def connect_to_spotify():
    print("To connect to the Spotify API, please provide your API credentials.")
    
    client_id = input("Enter your Spotify Client ID: ").strip()
    client_secret = input("Enter your Spotify Client Secret: ").strip()
    redirect_uri = input("Enter your Redirect URI (as configured in the Spotify Developer Dashboard): ").strip()
    
    # Set up Spotify OAuth
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
        client_id=client_id,
        client_secret=client_secret,
        redirect_uri=redirect_uri,
        scope="user-library-read"  # Adjust scopes as needed
    ))
    
    try:
        # Check connection by fetching user's library
        results = sp.current_user_saved_tracks(limit=1)
        print("Successfully connected to Spotify API!")
        print(f"First saved track: {results['items'][0]['track']['name']} by {results['items'][0]['track']['artists'][0]['name']}")
    except spotipy.exceptions.SpotifyException as e:
        print("Failed to connect to Spotify API. Please check your credentials.")
        print("Error:", e)

connect_to_spotify()

In [ ]:
def get_audio_features(sp, track_ids):
    # Get audio features for each track
    audio_features = sp.audio_features(track_ids)
    return audio_features

def recommend_tracks_by_mood(sp, mood, track_limit=5):
    # Define mood-based criteria for filtering songs
    mood_criteria = {
        'happy': {'valence': (0.7, 1.0), 'danceability': (0.6, 1.0), 'energy': (0.6, 1.0)},
        'sad': {'valence': (0.0, 0.4), 'energy': (0.0, 0.4)},
        'relaxed': {'valence': (0.4, 0.7), 'energy': (0.3, 0.6), 'acousticness': (0.5, 1.0)},
        'energetic': {'energy': (0.7, 1.0), 'danceability': (0.7, 1.0), 'tempo': (120, 180)}
    }

    if mood not in mood_criteria:
        print("Mood not recognized. Please choose from 'happy', 'sad', 'relaxed', or 'energetic'.")
        return

    # Fetch user's saved tracks and filter by mood
    saved_tracks = sp.current_user_saved_tracks(limit=50)  # Fetch up to 50 saved tracks
    track_ids = [item['track']['id'] for item in saved_tracks['items']]
    
    # Get audio features for these tracks
    audio_features = get_audio_features(sp, track_ids)

    # Filter tracks based on mood criteria
    matching_tracks = []
    criteria = mood_criteria[mood]
    
    for feature, track in zip(audio_features, saved_tracks['items']):
        if feature:  # Ensure the audio features exist for the track
            match = all(criteria[key][0] <= feature[key] <= criteria[key][1] for key in criteria)
            if match:
                matching_tracks.append(track['track'])
    
    # Randomly sample and display the matching tracks
    if matching_tracks:
        print(f"\nTracks matching the '{mood}' mood:")
        for track in random.sample(matching_tracks, min(track_limit, len(matching_tracks))):
            print(f"- {track['name']} by {track['artists'][0]['name']}")
    else:
        print(f"No tracks found for the '{mood}' mood in your saved tracks.")

# Connect to Spotify
sp = connect_to_spotify()

# Prompt user for mood selection and recommend tracks
user_mood = input("Enter your mood (options: 'happy', 'sad', 'relaxed', 'energetic'): ").strip().lower()
recommend_tracks_by_mood(sp, user_mood)